# Import Packages

In [21]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# Load Data

In [22]:
df = pd.read_csv(
    'analysis_data/location.csv',
    dtype = {"start_stn_code": "object"}
)

In [23]:
df.head()

,yyyymm,yyyy,mmm,trip_length,avg_dur_sec,rides,is_member,start_stn_code,stn_lat,stn_lon,stn_name
0,2014-04,2014,Apr,11-20 min,885.4242,33,0,6221,45.519410,-73.586850,du Mont-Royal / Clark
1,2014-04,2014,Apr,11-20 min,983.7500,4,0,6225,45.520188,-73.590559,Villeneuve / St-Urbain
2,2014-04,2014,Apr,11-20 min,851.9091,11,0,6232,45.525021,-73.610737,Hutchison / Van Horne
3,2014-04,2014,Apr,11-20 min,844.4583,24,0,6233,45.524296,-73.604847,Bernard / Jeanne-Mance
4,2014-04,2014,Apr,11-20 min,842.8077,26,0,6235,45.526543,-73.598233,St-Dominique / St-Viateur


# Plot 1: Annual Ridership

In [297]:
###########################
#PLOT 1: Annual Ridership #
###########################

## Create plotting dataframe from base dataframe
# Group by year
plot_df = df.groupby(
    by = ['yyyy','is_member'],
    as_index = False
).agg(
    rides = ('rides','sum')
)

# Pivot by is member or not
plot_df = plot_df.pivot(columns = 'is_member',
                  index = 'yyyy',
                  values = 'rides')

# Create new columns to calculate total rides and % membership
plot_df['total'] = plot_df[0] + plot_df[1]
plot_df['membership_pct'] = plot_df[1] / plot_df['total']


## Create plotly figure
# Initiate figure object
fig = go.Figure()

# Specify common x axis values for traces
x = plot_df.index.values

# Trace 1: Total rides, create hover template and bar chart
hovertemplate = "<br>".join(
    [
        "Total trips: %{y:.2f} M",
        "<extra></extra>"
    ]
)

texttemplate = "<br>".join(
    [
        "%{y:.2f}"
    ]
)

fig.add_trace(
        go.Bar(
            name = "Total trips",
            x = x,
            y = plot_df['total'] / 1_000_000, # Numbers in millions of rides
            hovertemplate = hovertemplate,
            texttemplate = texttemplate,
            marker_color = "cornflowerblue"
        )
)

# Trace 2: Member rides, create hover template and bar chart
# Create customdata to store % values for display
customdata = np.transpose([x,plot_df['membership_pct'].tolist()])
hovertemplate = "<br>".join(
    [
        "by Members: %{y:.2f} M",
        "% of year: %{customdata[1]:.2%}",
        "<extra></extra>"
    ]
)

texttemplate = "<br>".join(
    [
        "%{y:.2f}"
    ]
)

fig.add_trace(
        go.Bar(
            name = "by Members",
            x = x,
            y = plot_df[1]/ 1_000_000,
            customdata = customdata,
            hovertemplate = hovertemplate,
            texttemplate = texttemplate,
            marker_color = "cornflowerblue",
            marker_pattern_shape = "/"
        )
)

# Trace 3: Non-member rides, create hover template and bar chart
customdata = np.transpose([x,[1 - val for val in plot_df['membership_pct'].tolist()]])
hovertemplate = "<br>".join(
    [
        "by Non-members: %{y:.2f} M",
        "% of year: %{customdata[1]:.2%}",
        "<extra></extra>"
    ]
)

texttemplate = "<br>".join(
    [
        "%{y:.2f}"
    ]
)

fig.add_trace(
        go.Bar(
            name = "by Non-members",
            x = x,
            y = plot_df[0]/ 1_000_000,
            customdata = customdata,
            hovertemplate = hovertemplate,
            texttemplate = texttemplate,
            marker_color = "cornflowerblue",
            marker_pattern_shape = "+"
        )
)

# Update the display settings for text on bar chart
fig.update_traces(
    textfont_size = 12,
    textangle = 90,
    textposition = 'outside',
    # The cliponaxis attribute is set to False in the example below to ensure that the outside text on the tallest bar is allowed to render outside of the plotting area
    cliponaxis = False,
)

## Format x, y axis titles, title
fig.update_layout(
    barmode = 'group',
    hovermode = "x unified",
    # margin=dict(l=0, r=0, t=0, b=0),
    
    # Set uniform minimum textsize and force show
    # For max size, refer to textfont_size in update_traces
    uniformtext = dict(
        minsize = 9,
        mode = 'show'
    ),

    # Set title properties
     title = dict(
         text = f"Number of Bixi Trips by Year",
         x = 0.5,
         y = 0.85,
         xanchor = 'center',
         yanchor = 'top'
     ),

    # Set xaxis properties
    xaxis = dict(
        title = "Year",
        #titlefont_size = 14,
        tickfont_size = 12,
        #tickangle = -90,
        tickvals = x
    ),
    # Set yaxis properties
    yaxis = dict(
        title = f"Number of Bixi Trips (millions)",
        titlefont_size = 14,
        tickfont_size = 12
    )
    
)

#################
# END OF PLOT 1 #
#################

fig.show()

# Plot 2: Monthly Ridership

In [298]:
############################
# PLOT 2 Monthly Ridership #
############################

plot_df = df.groupby(
    by = ['yyyymm','yyyy','mmm'],
    as_index = False).agg(
    rides = ('rides','sum'))

## Initiate plotly graph object
fig = go.Figure()

x = plot_df['mmm'].unique()

# Add a line for each year 
for year in plot_df['yyyy'].unique():

    x = plot_df.loc[plot_df['yyyy'] == year, "mmm"].unique()
    y = plot_df.loc[plot_df['yyyy'] == year, "rides"]/1_000
    pct_yr_total = [val/y.sum() for val in y]
    customdata = np.transpose([x,pct_yr_total])
    
    hovertemplate = "<b>%{text} </b>" + "Trips: %{y:,.0f} K" + ",  % of year: %{customdata[1]:.2%}" + "<extra></extra>"
    
    fig.add_trace(
        go.Scatter(
            name = str(year), 
            x = x, 
            y = y,
            # mode = "lines+markers+text",
            hovertemplate = hovertemplate,
            customdata = customdata,
            text = [year] * len(x),
        )
    )

fig.update_layout(
    barmode = 'group',
    hovermode = "x unified",
    
    # Set uniform minimum textsize and force show
    # For max size, refer to textfont_size in update_traces
    uniformtext = dict(
        minsize = 9,
        mode = 'show'
    ),

    # Set title properties
     title = dict(
         text = f"Number of Bixi Trips by Month",
         x = 0.5,
         y = 0.85,
         xanchor = 'center',
         yanchor = 'top'
     ),

    # Set xaxis properties
    xaxis = dict(
        title = "Month",
        #titlefont_size = 14,
        tickfont_size = 12,
        #tickangle = -90,
        tickvals = x,
        side = 'bottom'
    ),
    # Set yaxis properties
    yaxis = dict(
        title = f"Number of Bixi Trips (thousands)",
        titlefont_size = 14,
        tickfont_size = 12
    )
)

#################
# END OF PLOT 2 #
#################

fig.show()

# Plot 3: Trip Duration binned

In [29]:
########################
# PLOT 3 Trip Duration #
########################

# df = pd.read_csv('analysis_data/ride_length.csv')

plot_df = df.groupby(
    by = ['yyyy','trip_length'],
    as_index = False).agg(
        rides = ('rides','sum')
)

# Initiate plotly graph object
fig = go.Figure()

# Define trip duration groups for x axis
x = plot_df['trip_length'].unique()

# Add a bar for each year
for year in plot_df['yyyy'].unique():

    x = plot_df.loc[plot_df['yyyy'] == year, "trip_length"].unique()
    y = plot_df.loc[plot_df['yyyy'] == year, "rides"]/1_000_000
    pct_yr_total = [val/y.sum() for val in y]
    customdata = np.transpose([x,pct_yr_total])

    hovertemplate = "<b>%{text} </b>" + "Trips: %{y:,.2f} M" + ",  % of year: %{customdata[1]:.2%}" + "<extra></extra>"

    texttemplate = "<br>".join(
        [
            "%{y:.2f}"
        ]
    )


    fig.add_trace(
            go.Bar(
                name = str(year),
                x = x,
                y = y, # Numbers in millions of rides,
                hovertemplate = hovertemplate,
                customdata = customdata,
                texttemplate = texttemplate,
                text = [year] * len(x)
            )
    )

fig.update_layout(
    barmode = 'group',
    hovermode = 'x unified',

    # Set uniform minimum textsize and force show
    # For max size, refer to textfont_size in update_traces
    uniformtext = dict(
        minsize = 9,
        mode = 'show'
    ),

    # Set title properties
     title = dict(
         text = f"Number of Trips by Trip Duration",
         x = 0.5,
         y = 0.85,
         xanchor = 'center',
         yanchor = 'top'
     ),

    # Set xaxis properties
    xaxis = dict(
        title = "Trip Duration (min)",
        #titlefont_size = 14,
        tickfont_size = 12,
        #tickangle = -90,
        tickvals = x,
        side = 'bottom'
    ),
    # Set yaxis properties
    yaxis = dict(
        title = f"Number of Bixi Trips (millions)",
        titlefont_size = 14,
        tickfont_size = 12
    )
)


#################
# END OF PLOT 3 #
#################

fig.show()

# Plot 4: Stations

In [41]:
groupby_clause = ['yyyy','start_stn_code','stn_lat','stn_lon']

plot_df = df.groupby(
    by = groupby_clause).agg(     
        rides = ('rides','sum'),
        # stn_lat = ('stn_lat','mean'),
        # stn_lon = ('stn_lon','mean'),
        stn_name = ('stn_name', lambda x: x.iloc[-1])
    )    

plot_df['avg_dur_sec'] = df.groupby(
    by = groupby_clause).apply(lambda x: np.average(x.avg_dur_sec, weights = x.rides))

plot_df.reset_index(inplace = True)

plot_df.rename(
    columns = 
    {
        "yyyy"        : "Year",
        "stn_lat"     : "Lat",
        "stn_lon"     : "Lon",
        "rides"       : "Trips",
        "stn_name"    : "Station Name",
        "avg_dur_sec" : "Avg. Trip Duration (min)"
    },
    inplace = True
)

plot_df['Avg. Trip Duration (min)'] = plot_df['Avg. Trip Duration (min)']/60 

px.set_mapbox_access_token(open(".mapbox_token").read())

fig = px.scatter_mapbox(
    plot_df,
    lat = "Lat",
    lon = "Lon",
    text = "Station Name",
    title = "Number of Bixi Trips by Station",
    hover_name = "Station Name",
    hover_data = {
        "Trips"                    : ":,",
        "Lat"                      : ":.3f",
        "Lon"                      : ":.3f",
        "Avg. Trip Duration (min)" : ":,.0f",
        "Station Name"             : False
    },
    size = "Trips",
    size_max = 15,
    color = "Avg. Trip Duration (min)",
    color_continuous_scale = "Oryel",
    zoom = 11,
    mapbox_style = 'basic',
    animation_frame= 'Year')

fig.layout.coloraxis.colorbar.title.text = "Avg. Trip<br>Dur. (min)"
fig.update_traces(
    marker_colorbar_orientation = "h")

fig.show()